In [3]:
# prompt: connect with google drive to a specific folder

from google.colab import drive
drive.mount('/content/drive')

# Get the folder ID from the URL of the desired folder in Google Drive.
# For example, if the URL is "https://drive.google.com/drive/folders/1234567890",
# then the folder ID is "1234567890".
folder_id = 'examenes'

# Create a path to the desired folder.
folder_path = f'/content/drive/MyDrive/{folder_id}'

# Change the current working directory to the desired folder.
%cd {folder_path}

Mounted at /content/drive
/content/drive/MyDrive/examenes


# importing dataframe

In [79]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import balanced_accuracy_score
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.ensemble import RandomForestClassifier
!pip install catboost
from catboost import CatBoostClassifier

In [64]:
df=pd.read_csv("hr_ibm.csv")

# EDA

In [65]:
df.dtypes

Age                          int64
Attrition                   object
BusinessTravel              object
DailyRate                    int64
Department                  object
DistanceFromHome             int64
Education                    int64
EducationField              object
EmployeeCount                int64
EmployeeNumber               int64
EnvironmentSatisfaction      int64
Gender                      object
HourlyRate                   int64
JobInvolvement               int64
JobLevel                     int64
JobRole                     object
JobSatisfaction              int64
MaritalStatus               object
MonthlyIncome                int64
MonthlyRate                  int64
NumCompaniesWorked           int64
Over18                      object
OverTime                    object
PercentSalaryHike            int64
PerformanceRating            int64
RelationshipSatisfaction     int64
StandardHours                int64
StockOptionLevel             int64
TotalWorkingYears   

In [66]:
df=df.drop_duplicates()
df= df.dropna()
df.isna().sum()

Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSince

In [67]:
len(df)

1470

In [44]:
#This function is to remove outliers. However, by using it, the dataset length is reduced from 1470 to 699
#def remove_outliers(df, col):
#  Q1 = df[col].quantile(0.25)
#  Q3 = df[col].quantile(0.75)
#  IQR = Q3 - Q1
#  lower_bound = Q1 - 1.5 * IQR
#  upper_bound = Q3 + 1.5 * IQR
#  df_filtered = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
#  return df_filtered

#for col in df.select_dtypes(include=["int", "float"]):
#  df = remove_outliers(df, col)

# Pre-processing

In [68]:
encoder = LabelEncoder()
columns_to_encode=["Attrition", "BusinessTravel", "Department", "EducationField", "Gender", "JobRole", "MaritalStatus", "Over18", "OverTime"]
for column in columns_to_encode:
  encoder.fit(df[column])
  df[column] = encoder.transform(df[column])

In [74]:
scaler = MinMaxScaler()
df_normalized = scaler.fit_transform(df)
df_normalized = pd.DataFrame(df_normalized, columns=df.columns)

# Train-Test

In [75]:
df_normalized

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0.547619,1.0,1.0,0.715820,1.0,0.000000,0.25,0.2,0.0,0.000000,...,0.000000,0.0,0.000000,0.200,0.000000,0.000000,0.150,0.222222,0.000000,0.294118
1,0.738095,0.0,0.5,0.126700,0.5,0.250000,0.00,0.2,0.0,0.000484,...,1.000000,0.0,0.333333,0.250,0.500000,0.666667,0.250,0.388889,0.066667,0.411765
2,0.452381,1.0,1.0,0.909807,0.5,0.035714,0.25,0.8,0.0,0.001451,...,0.333333,0.0,0.000000,0.175,0.500000,0.666667,0.000,0.000000,0.000000,0.000000
3,0.357143,0.0,0.5,0.923407,0.5,0.071429,0.75,0.2,0.0,0.001935,...,0.666667,0.0,0.000000,0.200,0.500000,0.666667,0.200,0.388889,0.200000,0.000000
4,0.214286,0.0,1.0,0.350036,0.5,0.035714,0.00,0.6,0.0,0.002903,...,1.000000,0.0,0.333333,0.150,0.500000,0.666667,0.050,0.111111,0.133333,0.117647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,0.428571,0.0,0.5,0.559771,0.5,0.785714,0.25,0.6,0.0,0.996613,...,0.666667,0.0,0.333333,0.425,0.500000,0.666667,0.125,0.111111,0.000000,0.176471
1466,0.500000,0.0,1.0,0.365784,0.5,0.178571,0.00,0.6,0.0,0.997097,...,0.000000,0.0,0.333333,0.225,0.833333,0.666667,0.175,0.388889,0.066667,0.411765
1467,0.214286,0.0,1.0,0.037938,0.5,0.107143,0.50,0.2,0.0,0.998065,...,0.333333,0.0,0.333333,0.150,0.000000,0.666667,0.150,0.111111,0.000000,0.176471
1468,0.738095,0.0,0.5,0.659270,1.0,0.035714,0.50,0.6,0.0,0.998549,...,1.000000,0.0,0.000000,0.425,0.500000,0.333333,0.225,0.333333,0.000000,0.470588


In [78]:
df_features=df_normalized.drop(columns=["Attrition"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(df_features, df_normalized['Attrition'], test_size=0.2)


# Model 1: Decision Tree

In [80]:
max_depths = [None, 5, 10, 20, 50, 100]

accuracies = []

for max_depth in max_depths:
    clf = DecisionTreeClassifier(random_state=42, max_depth=max_depth)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

best_max_depth = max_depths[np.argmax(accuracies)]

dt = DecisionTreeClassifier(random_state=42, max_depth=best_max_depth)

dt.fit(X_train, y_train)

y_pred_dt = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_dt))
print("\nClassification Report:\n", classification_report(y_test, y_pred_dt))

Accuracy: 0.7721088435374149

Classification Report:
               precision    recall  f1-score   support

         0.0       0.86      0.86      0.86       244
         1.0       0.33      0.32      0.32        50

    accuracy                           0.77       294
   macro avg       0.59      0.59      0.59       294
weighted avg       0.77      0.77      0.77       294



# Model 2: Logistic Regression

In [83]:
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)


y_pred_lr = lr.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print("\nClassification Report:\n", classification_report(y_test, y_pred_lr))

Accuracy: 0.8571428571428571

Classification Report:
               precision    recall  f1-score   support

         0.0       0.87      0.97      0.92       244
         1.0       0.68      0.30      0.42        50

    accuracy                           0.86       294
   macro avg       0.78      0.64      0.67       294
weighted avg       0.84      0.86      0.83       294



# Model 3: Random Forest

In [84]:
# Initialize and train a Random Forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("\nClassification Report:\n", classification_report(y_test, y_pred_rf))

Accuracy: 0.8435374149659864

Classification Report:
               precision    recall  f1-score   support

         0.0       0.84      1.00      0.91       244
         1.0       0.83      0.10      0.18        50

    accuracy                           0.84       294
   macro avg       0.84      0.55      0.55       294
weighted avg       0.84      0.84      0.79       294



# Model 4: XGBoost

In [85]:
xgb_clf = xgb.XGBClassifier(objective='multi:softmax',
                            num_class=3,
                            missing=1,
                            early_stopping_rounds=10,
                            eval_metric=['merror','mlogloss'],
                            seed=42)
xgb_clf.fit(X_train,
            y_train,
            verbose=0, # set to 1 to see xgb training round intermediate results
            eval_set=[(X_train, y_train), (X_test, y_test)])


results = xgb_clf.evals_result()
epochs = 1
x_axis = range(0, epochs)

y_pred_xg = xgb_clf.predict(X_test)
print("Accuracy:", balanced_accuracy_score(y_test, y_pred_xg))
print("\nClassification Report:\n", classification_report(y_test, y_pred_xg))

Accuracy: 0.5515573770491804

Classification Report:
               precision    recall  f1-score   support

         0.0       0.85      0.96      0.90       244
         1.0       0.44      0.14      0.21        50

    accuracy                           0.82       294
   macro avg       0.64      0.55      0.56       294
weighted avg       0.78      0.82      0.78       294



# Model 5: Catboost

In [92]:
model = CatBoostClassifier(iterations=5000,
                           learning_rate=0.05,
                           depth=10,
                           loss_function='Logloss',
                           eval_metric='Accuracy',
                           l2_leaf_reg=3)

model.fit(X_train, y_train,
         eval_set=(X_test, y_test),
         verbose=500,
         plot=True)

y_pred_cb = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_cb))
print("\nClassification Report:\n", classification_report(y_test, y_pred_cb))


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.8460884	test: 0.8197279	best: 0.8197279 (0)	total: 50ms	remaining: 4m 9s
500:	learn: 1.0000000	test: 0.8605442	best: 0.8605442 (492)	total: 24.9s	remaining: 3m 43s
1000:	learn: 1.0000000	test: 0.8571429	best: 0.8605442 (492)	total: 50.4s	remaining: 3m 21s
1500:	learn: 1.0000000	test: 0.8605442	best: 0.8605442 (492)	total: 1m 14s	remaining: 2m 54s
2000:	learn: 1.0000000	test: 0.8605442	best: 0.8605442 (492)	total: 1m 46s	remaining: 2m 39s
2500:	learn: 1.0000000	test: 0.8605442	best: 0.8605442 (492)	total: 2m 5s	remaining: 2m 5s
3000:	learn: 1.0000000	test: 0.8537415	best: 0.8605442 (492)	total: 2m 20s	remaining: 1m 33s
3500:	learn: 1.0000000	test: 0.8571429	best: 0.8605442 (492)	total: 2m 43s	remaining: 1m 10s
4000:	learn: 1.0000000	test: 0.8571429	best: 0.8605442 (492)	total: 3m	remaining: 45s
4500:	learn: 1.0000000	test: 0.8571429	best: 0.8605442 (492)	total: 3m 21s	remaining: 22.4s
4999:	learn: 1.0000000	test: 0.8571429	best: 0.8605442 (492)	total: 3m 40s	remaining: 0us



# Conclusions

Considering the five developed models, Catboost is the one that better predict the outcomes. However, dataset is imbalanced. For this reason, one category is better predicted.